In [2]:
import psutil
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os
import pandas as pd
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor
import threading
from queue import Queue

class AdvancedCPUAnalyzer:
    def __init__(self, duration=60):
        self.duration = duration
        self.data = {
            'timestamps': [],
            'cache_metrics': [],
            'parallel_metrics': [],
            'cpu_usage': [],
            'ram_usage': [],
            'cpi': [],
            'ipc': [],
            'temperature': [],
            'power_draw': [],
            'thread_usage': []
        }
        self.stop_monitoring = False
        self.threads = psutil.cpu_count()
        self.cores = psutil.cpu_count(logical=False)
        self.thread_pool = ThreadPoolExecutor(max_workers=self.threads)
        self.parallel_queue = Queue()

    def collect_system_metrics(self):
        """Collects CPU, RAM, CPI, IPC, temperature, and power metrics"""
        cpu_usage = psutil.cpu_percent(interval=1)
        ram_usage = psutil.virtual_memory().percent
        # Placeholder values for CPI, IPC, temperature, and power draw
        cpi = 1.5
        ipc = 0.66
        temperature = 42.3  
        power_draw = 22.1  
        thread_usage = np.mean([usage for usage in psutil.cpu_percent(percpu=True)])

        return {
            'cpu_usage': cpu_usage,
            'ram_usage': ram_usage,
            'cpi': cpi,
            'ipc': ipc,
            'temperature': temperature,
            'power_draw': power_draw,
            'thread_usage': thread_usage
        }

    def simulate_cache_hierarchy(self):
        """Simulate detailed cache hierarchy performance"""
        cpu_load = psutil.cpu_percent()
        cache_sizes = {'L1': 32, 'L2': 256, 'L3': 12288}
        load_factor = cpu_load / 100
        cache_metrics = {
            'L1': {
                'hits': int(10000 * (0.95 - (load_factor * 0.05))),
                'misses': int(10000 * (0.05 + (load_factor * 0.05))),
                'latency_ns': 1,
                'bandwidth_gbps': 700 - (load_factor * 100)
            },
            'L2': {
                'hits': int(8000 * (0.85 - (load_factor * 0.07))),
                'misses': int(8000 * (0.15 + (load_factor * 0.07))),
                'latency_ns': 4,
                'bandwidth_gbps': 200 - (load_factor * 50)
            },
            'L3': {
                'hits': int(5000 * (0.75 - (load_factor * 0.1))),
                'misses': int(5000 * (0.25 + (load_factor * 0.1))),
                'latency_ns': 12,
                'bandwidth_gbps': 100 - (load_factor * 25)
            }
        }
        for level in cache_metrics:
            total_accesses = cache_metrics[level]['hits'] + cache_metrics[level]['misses']
            cache_metrics[level]['hit_rate'] = cache_metrics[level]['hits'] / total_accesses
            cache_metrics[level]['size_kb'] = cache_sizes[level]
            cache_metrics[level]['utilization'] = (cpu_load / 100) * 0.8 + 0.2
        return cache_metrics

    def measure_parallel_performance(self):
        """Measure parallel processing performance metrics"""
        def parallel_test_task():
            start = time.time()
            sum([i * i for i in range(1000000)])
            return time.time() - start

        single_thread_time = parallel_test_task()
        with ThreadPoolExecutor(max_workers=self.threads) as executor:
            start = time.time()
            list(executor.map(lambda x: parallel_test_task(), range(self.threads)))
            multi_thread_time = time.time() - start

        speedup = single_thread_time / (multi_thread_time / self.threads)
        efficiency = speedup / self.threads

        return {
            'speedup': speedup,
            'parallel_efficiency': efficiency,
            'core_scaling': speedup / self.cores,
            'thread_scaling': speedup / self.threads,
            'active_threads': len([x for x in psutil.cpu_percent(percpu=True) if x > 10]),
            'thread_imbalance': np.std(psutil.cpu_percent(percpu=True))
        }

    def collect_data(self):
        """Collect all performance metrics"""
        start_time = time.time()
        print("Collecting comprehensive performance data...")

        while time.time() - start_time < self.duration and not self.stop_monitoring:
            timestamp = datetime.now()
            cache_metrics = self.simulate_cache_hierarchy()
            parallel_metrics = self.measure_parallel_performance()
            system_metrics = self.collect_system_metrics()

            self.data['timestamps'].append(timestamp)
            self.data['cache_metrics'].append(cache_metrics)
            self.data['parallel_metrics'].append(parallel_metrics)
            self.data['cpu_usage'].append(system_metrics['cpu_usage'])
            self.data['ram_usage'].append(system_metrics['ram_usage'])
            self.data['cpi'].append(system_metrics['cpi'])
            self.data['ipc'].append(system_metrics['ipc'])
            self.data['temperature'].append(system_metrics['temperature'])
            self.data['power_draw'].append(system_metrics['power_draw'])
            self.data['thread_usage'].append(system_metrics['thread_usage'])

            time.sleep(1)

    def generate_report(self):
        """Generate comprehensive performance report"""
        os.makedirs('reports', exist_ok=True)
        with open('reports/detailed_performance_report.txt', 'w') as f:
            f.write("Advanced CPU, Cache, and Parallel Performance Analysis Report\n")
            f.write("=====================================================\n\n")
            f.write(f"System Information:\nCPU: AMD64 Family 23 Model 24 Stepping 1, AuthenticAMD\n")
            f.write(f"Cores/Threads: {self.cores} / {self.threads}\nOS: Windows 11\n")
            f.write("\nCPU Performance Metrics:\n")
            f.write(f"Average CPU Usage: {np.mean(self.data['cpu_usage']):.2f}%\n")
            f.write(f"Peak CPU Usage: {np.max(self.data['cpu_usage']):.2f}%\n")
            f.write("\nRAM Usage:\n")
            f.write(f"Average RAM Usage: {np.mean(self.data['ram_usage']):.2f}%\n")
            f.write("\nCPI/IPC Metrics:\n")
            f.write(f"Average CPI: {np.mean(self.data['cpi']):.2f}\n")
            f.write(f"Average IPC: {np.mean(self.data['ipc']):.2f}\n")
            f.write("\nThread Performance:\n")
            f.write(f"Average Thread Usage: {np.mean(self.data['thread_usage']):.2f}%\n")
            f.write("\nPower and Thermal Metrics:\n")
            f.write(f"Average Temperature: {np.mean(self.data['temperature']):.2f}°C\n")
            f.write(f"Average Power Draw: {np.mean(self.data['power_draw']):.2f}W\n")
            
        print("Performance report generated and saved to 'reports/detailed_performance_report.txt'.")

if __name__ == "__main__":
    analyzer = AdvancedCPUAnalyzer(duration=60)
    analyzer.collect_data()
    analyzer.generate_report()


Performance report generated and saved to 'reports/detailed_performance_report.txt'.
